In [1]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [3]:
#X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
#Y = np_plus_liability / df["Total Asset"]
Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [4]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,150.000000,150.000000
mean,1.326667,0.523041
std,1.217857,2.069191
min,0.000000,0.007332
25%,0.000000,0.100060
50%,1.000000,0.146972
75%,2.000000,0.284729
max,6.000000,18.006205


In [5]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [6]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7489
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.388
Time:                        20:04:11   Log-Likelihood:                -321.03
No. Observations:                 150   AIC:                             646.1
Df Residuals:                     148   BIC:                             652.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6830      0.250      2.726      0.007       0.188       1.178
X             -0.1206      0.139     -0.865      0.388      -0.396       0.155
==============================================================================
Omnibus:                      256.997   Durbin-Watson:                   0.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27230.546
Skew:                           7.971   Prob(JB):                         0.00
Kurtosis:                      67.053   Cond. No.                         3.17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [8]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     27.27
Date:                Thu, 11 May 2023   Prob (F-statistic):           5.89e-07
Time:                        20:04:11   Log-Likelihood:                -298.27
No. Observations:                 150   AIC:                             600.5
Df Residuals:                     148   BIC:                             606.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7893      0.215      3.667      0.000       0.364       1.215
X              0.6251      0.120      5.222      0.000       0.389       0.862
==============================================================================
Omnibus:                      263.387   Durbin-Watson:                   1.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30726.503
Skew:                           8.315   Prob(JB):                         0.00
Kurtosis:                      71.115   Cond. No.                         3.17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [10]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [11]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

70    6
71    6
72    6
Name: X, dtype: int64

In [12]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [13]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [14]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [15]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [16]:
data_sample.describe()

,X,Y
count,131.000000,131.000000
mean,1.328244,0.172250
std,1.026141,0.114009
min,0.000000,0.007332
25%,1.000000,0.096075
50%,1.000000,0.135867
75%,2.000000,0.207618
max,4.000000,0.576045


In [17]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     3.231
Date:                Thu, 11 May 2023   Prob (F-statistic):             0.0746
Time:                        20:04:12   Log-Likelihood:                 100.70
No. Observations:                 131   AIC:                            -197.4
Df Residuals:                     129   BIC:                            -191.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1492      0.016      9.212      0.000       0.117       0.181
X              0.0174      0.010      1.798      0.075      -0.002       0.036
==============================================================================
Omnibus:                       29.359   Durbin-Watson:                   0.923
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.298
Skew:                           1.253   Prob(JB):                     1.78e-09
Kurtosis:                       4.048   Cond. No.                         3.44
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
bX = data_sample["X"] * result.params["X"]

In [19]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [20]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.8809
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.350
Time:                        20:04:12   Log-Likelihood:                 160.61
No. Observations:                 131   AIC:                            -317.2
Df Residuals:                     129   BIC:                            -311.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0790      0.010      7.709      0.000       0.059       0.099
X              0.0057      0.006      0.939      0.350      -0.006       0.018
==============================================================================
Omnibus:                       51.310   Durbin-Watson:                   1.215
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              127.903
Skew:                           1.582   Prob(JB):                     1.68e-28
Kurtosis:                       6.663   Cond. No.                         3.44
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""